In [7]:
from funciones_analisis import obtener_asistencias
import datetime
import pandas as pd
import numpy as np

In [8]:
asistencias = obtener_asistencias()

# Crear el DataFrame de pandas
df = pd.DataFrame(asistencias, columns=['id_asistencia', 'fecha', 'hora_entrada', 'hora_salida', 'nombre', 'apellido', 'puesto', 'departamento'])
# Calcular horas trabajadas (como entero)
df['horas_trabajadas'] = (df['hora_salida'] - df['hora_entrada']).apply(lambda x: int(x.total_seconds() // 3600))
# Convertir las columnas de hora_entrada y hora_salida a formato de hora, minuto, segundo
df['hora_entrada'] = df['hora_entrada'].apply(lambda x: str(x).split()[2])  # Obtener solo la parte de la hora
df['hora_salida'] = df['hora_salida'].apply(lambda x: str(x).split()[2])  # Obtener solo la parte de la hora
df = df.sort_values(by='id_asistencia').reset_index(drop=True)
df.head()



,id_asistencia,fecha,hora_entrada,hora_salida,nombre,apellido,puesto,departamento,horas_trabajadas
0,15,2025-01-01,06:00:00,14:00:00,miguel,martinez,Maestro Repostero,Cocina,8
1,16,2025-01-01,06:00:00,14:00:00,Carlos Alberto,Ramírez González,Maestro Repostero,Cocina,8
2,17,2025-01-01,07:00:00,15:00:00,María Fernanda,González Pérez,Ayudante de Repostería,Cocina,8
3,18,2025-01-01,07:00:00,15:00:00,José Antonio,Hernández López,Ayudante de Repostería,Cocina,8
4,19,2025-01-01,07:00:00,15:00:00,Ana María,López Rodríguez,Ayudante de Repostería,Cocina,8


In [9]:
# Asegurarse de que 'fecha' sea tipo datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Agrupar por nombre, apellido y puesto por semana
horas_por_semana = df.set_index('fecha').groupby(['nombre', 'apellido', 'puesto', pd.Grouper(freq='W')])['horas_trabajadas'].sum().reset_index()

# Agrupar por mes
horas_por_mes = df.set_index('fecha').groupby(['nombre', 'apellido', 'puesto', pd.Grouper(freq='M')])['horas_trabajadas'].sum().reset_index()

# Crear columna de quincena
df['quincena'] = df['fecha'].apply(lambda x: f"{x.year}-{x.month:02d}-Q1" if x.day <= 15 else f"{x.year}-{x.month:02d}-Q2")

# Agrupar por quincena
horas_por_quincena = df.groupby(['nombre', 'apellido', 'puesto', 'quincena'])['horas_trabajadas'].sum().reset_index()

# Mostrar resultados
print("Horas por semana:")
print(horas_por_semana)

print("\nHoras por quincena:")
print(horas_por_quincena)

print("\nHoras por mes:")
print(horas_por_mes)


Horas por semana:
            nombre         apellido                  puesto      fecha  \
0        Ana María  López Rodríguez  Ayudante de Repostería 2025-01-05   
1        Ana María  López Rodríguez  Ayudante de Repostería 2025-01-12   
2        Ana María  López Rodríguez  Ayudante de Repostería 2025-01-19   
3        Ana María  López Rodríguez  Ayudante de Repostería 2025-01-26   
4        Ana María  López Rodríguez  Ayudante de Repostería 2025-02-02   
..             ...              ...                     ...        ...   
177  pablo tercero      pineda cruz   Encargado de Limpieza 2025-03-09   
178  pablo tercero      pineda cruz   Encargado de Limpieza 2025-03-16   
179  pablo tercero      pineda cruz   Encargado de Limpieza 2025-03-23   
180  pablo tercero      pineda cruz   Encargado de Limpieza 2025-03-30   
181  pablo tercero      pineda cruz   Encargado de Limpieza 2025-04-06   

     horas_trabajadas  
0                  16  
1                  47  
2                  47

C:\Users\VICTOR\AppData\Local\Temp\ipykernel_22468\28775759.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  horas_por_mes = df.set_index('fecha').groupby(['nombre', 'apellido', 'puesto', pd.Grouper(freq='M')])['horas_trabajadas'].sum().reset_index()


In [14]:
import pandas as pd

# Datos actualizados con departamento
datos = {
    "puesto": [
        "Maestro Repostero",
        "Ayudante de Repostería",
        "Vendedor Mostrador",
        "Repartidor",
        "Encargado Administrativo",
        "Community Manager",
        "Encargado de Limpieza"
    ],
    "departamento": [
        "Cocina",
        "Cocina",
        "Ventas",
        "Ventas",
        "Oficina",
        "Marketing",
        "Limpieza"
    ],
    "pago_hora": [80, 50, 55, 50, 70, 60, 45]
}

# Crear DataFrame
df_puestos = pd.DataFrame(datos)

# Mostrar la tabla
print(df_puestos)


                     puesto departamento  pago_hora
0         Maestro Repostero       Cocina         80
1    Ayudante de Repostería       Cocina         50
2        Vendedor Mostrador       Ventas         55
3                Repartidor       Ventas         50
4  Encargado Administrativo      Oficina         70
5         Community Manager    Marketing         60
6     Encargado de Limpieza     Limpieza         45


In [19]:
# Unir por puesto para SEMANA
ingresos_por_semana = horas_por_semana.merge(df_puestos, on='puesto')
ingresos_por_semana['ingreso_semana'] = ingresos_por_semana['horas_trabajadas'] * ingresos_por_semana['pago_hora']

# Unir por puesto para QUINCENA
ingresos_por_quincena = horas_por_quincena.merge(df_puestos, on='puesto')
ingresos_por_quincena['ingreso_quincena'] = ingresos_por_quincena['horas_trabajadas'] * ingresos_por_quincena['pago_hora']

# Unir por puesto para MES
ingresos_por_mes = horas_por_mes.merge(df_puestos, on='puesto')
ingresos_por_mes['ingreso_mes'] = ingresos_por_mes['horas_trabajadas'] * ingresos_por_mes['pago_hora']

# Mostrar resultados
print("Ingresos por semana:")
ingresos_por_semana.head(10)
# print(ingresos_por_semana[['nombre', 'apellido', 'puesto', 'fecha', 'horas_trabajadas', 'ingreso_semana']])

# print("\nIngresos por quincena:")
# print(ingresos_por_quincena[['nombre', 'apellido', 'puesto', 'quincena', 'horas_trabajadas', 'ingreso_quincena']])

# print("\nIngresos por mes:")
# print(ingresos_por_mes[['nombre', 'apellido', 'puesto', 'fecha', 'horas_trabajadas', 'ingreso_mes']])


Ingresos por semana:


,nombre,apellido,puesto,fecha,horas_trabajadas,departamento,pago_hora,ingreso_semana
0,Ana María,López Rodríguez,Ayudante de Repostería,2025-01-05,16,Cocina,50,800
1,Ana María,López Rodríguez,Ayudante de Repostería,2025-01-12,47,Cocina,50,2350
2,Ana María,López Rodríguez,Ayudante de Repostería,2025-01-19,47,Cocina,50,2350
3,Ana María,López Rodríguez,Ayudante de Repostería,2025-01-26,44,Cocina,50,2200
4,Ana María,López Rodríguez,Ayudante de Repostería,2025-02-02,44,Cocina,50,2200
5,Ana María,López Rodríguez,Ayudante de Repostería,2025-02-09,47,Cocina,50,2350
6,Ana María,López Rodríguez,Ayudante de Repostería,2025-02-16,45,Cocina,50,2250
7,Ana María,López Rodríguez,Ayudante de Repostería,2025-02-23,47,Cocina,50,2350
8,Ana María,López Rodríguez,Ayudante de Repostería,2025-03-02,47,Cocina,50,2350
9,Ana María,López Rodríguez,Ayudante de Repostería,2025-03-09,55,Cocina,50,2750


In [20]:
print("Ingresos por quincena:")
ingresos_por_quincena.head(10)

Ingresos por quincena:


,nombre,apellido,puesto,quincena,horas_trabajadas,departamento,pago_hora,ingreso_quincena
0,Ana María,López Rodríguez,Ayudante de Repostería,2025-01-Q1,86,Cocina,50,4300
1,Ana María,López Rodríguez,Ayudante de Repostería,2025-01-Q2,104,Cocina,50,5200
2,Ana María,López Rodríguez,Ayudante de Repostería,2025-02-Q1,100,Cocina,50,5000
3,Ana María,López Rodríguez,Ayudante de Repostería,2025-02-Q2,86,Cocina,50,4300
4,Ana María,López Rodríguez,Ayudante de Repostería,2025-03-Q1,107,Cocina,50,5350
5,Ana María,López Rodríguez,Ayudante de Repostería,2025-03-Q2,91,Cocina,50,4550
6,Carlos Alberto,Ramírez González,Maestro Repostero,2025-01-Q1,84,Cocina,80,6720
7,Carlos Alberto,Ramírez González,Maestro Repostero,2025-01-Q2,107,Cocina,80,8560
8,Carlos Alberto,Ramírez González,Maestro Repostero,2025-02-Q1,98,Cocina,80,7840
9,Carlos Alberto,Ramírez González,Maestro Repostero,2025-02-Q2,84,Cocina,80,6720


In [22]:
print("\nIngresos por mes:")
ingresos_por_mes.head(10)


Ingresos por mes:


,nombre,apellido,puesto,fecha,horas_trabajadas,departamento,pago_hora,ingreso_mes
0,Ana María,López Rodríguez,Ayudante de Repostería,2025-01-31,190,Cocina,50,9500
1,Ana María,López Rodríguez,Ayudante de Repostería,2025-02-28,186,Cocina,50,9300
2,Ana María,López Rodríguez,Ayudante de Repostería,2025-03-31,198,Cocina,50,9900
3,Carlos Alberto,Ramírez González,Maestro Repostero,2025-01-31,191,Cocina,80,15280
4,Carlos Alberto,Ramírez González,Maestro Repostero,2025-02-28,182,Cocina,80,14560
5,Carlos Alberto,Ramírez González,Maestro Repostero,2025-03-31,196,Cocina,80,15680
6,Daniela Sofía,Morales Gómez,Community Manager,2025-01-31,191,Marketing,60,11460
7,Daniela Sofía,Morales Gómez,Community Manager,2025-02-28,188,Marketing,60,11280
8,Daniela Sofía,Morales Gómez,Community Manager,2025-03-31,206,Marketing,60,12360
9,Fernanda Isabel,Sánchez García,Repartidor,2025-01-31,190,Ventas,50,9500
